# A League of Legends Competitive Analysis

**Name(s)**: Kevin Wong, Andrew Yang

**Website Link**: https://kev374k.github.io/lol_competitive_analysis/

## Code

In [1]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
pd.options.plotting.backend = 'plotly'

### Baseline Model

In [2]:
df = pd.read_csv("2022_LoL_esports_match_data_from_OraclesElixir.csv")
df

/Users/kevinwong/anaconda3/envs/dsc80/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3505: DtypeWarning: Columns (2,76) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
0,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,121.0,391.0,345.0,14.0,0.0,1.0,0.0,0.0,1.0,0.0
1,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,100.0,541.0,-275.0,-11.0,2.0,3.0,2.0,0.0,5.0,1.0
2,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,119.0,-475.0,153.0,1.0,0.0,3.0,0.0,3.0,3.0,2.0
3,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,149.0,-793.0,-1343.0,-34.0,2.0,1.0,2.0,3.0,3.0,0.0
4,ESPORTSTMNT01_2690210,complete,NaN,LCKC,2022,Spring,0,2022-01-10 07:44:08,1,12.01,...,21.0,443.0,-497.0,7.0,1.0,2.0,2.0,0.0,6.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149395,9687-9687_game_5,partial,https://lpl.qq.com/es/stats.shtml?bmid=9687,DCup,2022,NaN,0,2022-12-27 12:43:43,5,12.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149396,9687-9687_game_5,partial,https://lpl.qq.com/es/stats.shtml?bmid=9687,DCup,2022,NaN,0,2022-12-27 12:43:43,5,12.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149397,9687-9687_game_5,partial,https://lpl.qq.com/es/stats.shtml?bmid=9687,DCup,2022,NaN,0,2022-12-27 12:43:43,5,12.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149398,9687-9687_game_5,partial,https://lpl.qq.com/es/stats.shtml?bmid=9687,DCup,2022,NaN,0,2022-12-27 12:43:43,5,12.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [221]:
# cleaning data: converts columns 'datacompleteness', 'playoffs', 'result'
# we then get the columns that are needed for our hypothesis
# Code is from our league of legends competitive analysis

cleaned_data = df.assign(
    datacompleteness = df['datacompleteness'].apply(lambda x: True if x == "complete" else False),
    playoffs = df['playoffs'].apply(lambda x: True if x == 1 else False),
    result = df['result'].apply(lambda x: True if x == 1 else False)
    ).loc[
        :, ['result', 'gameid', 'datacompleteness', 'playoffs', 'side', 'position', 'kills', 'deaths', 'assists', 'gamelength', 'dragons', 'barons', 'elders', 'heralds', 'cspm', 'wpm', 'vspm', 'golddiffat15', 'xpdiffat15']
    ].fillna(0)

In [222]:
cleaned_data = cleaned_data[cleaned_data['datacompleteness'] == True]

In [223]:
cleaned_data.head(12)

,result,gameid,datacompleteness,playoffs,side,position,kills,deaths,assists,firstblood,gamelength,dragons,barons,elders,heralds,cspm,wpm,vspm,golddiffat15,xpdiffat15
0,False,ESPORTSTMNT01_2690210,True,False,Blue,top,2,3,2,0.0,1713,0.0,0.0,0.0,0.0,8.0911,0.2802,0.9107,391.0,345.0
1,False,ESPORTSTMNT01_2690210,True,False,Blue,jng,2,5,6,1.0,1713,0.0,0.0,0.0,0.0,5.1839,0.2102,1.6813,541.0,-275.0
2,False,ESPORTSTMNT01_2690210,True,False,Blue,mid,2,2,3,0.0,1713,0.0,0.0,0.0,0.0,6.7601,0.6655,1.0158,-475.0,153.0
3,False,ESPORTSTMNT01_2690210,True,False,Blue,bot,2,4,2,1.0,1713,0.0,0.0,0.0,0.0,7.9159,0.4203,0.8757,-793.0,-1343.0
4,False,ESPORTSTMNT01_2690210,True,False,Blue,sup,1,5,6,1.0,1713,0.0,0.0,0.0,0.0,1.4711,1.0158,2.4168,443.0,-497.0
5,True,ESPORTSTMNT01_2690210,True,False,Red,top,1,1,12,0.0,1713,0.0,0.0,0.0,0.0,8.0210,0.5254,1.0508,-391.0,-345.0
6,True,ESPORTSTMNT01_2690210,True,False,Red,jng,4,1,10,0.0,1713,0.0,0.0,0.0,0.0,6.4098,0.2802,1.3660,-541.0,275.0
7,True,ESPORTSTMNT01_2690210,True,False,Red,mid,6,3,12,0.0,1713,0.0,0.0,0.0,0.0,7.5657,0.5254,1.0158,475.0,-153.0
8,True,ESPORTSTMNT01_2690210,True,False,Red,bot,8,2,10,0.0,1713,0.0,0.0,0.0,0.0,11.1734,0.3152,1.4011,793.0,1343.0
9,True,ESPORTSTMNT01_2690210,True,False,Red,sup,0,2,18,0.0,1713,0.0,0.0,0.0,0.0,1.0158,1.6112,2.3468,-443.0,497.0


### Baseline Model

Train a “baseline model” for your prediction task that uses at least two features. (For this requirement, two features means selecting at least two columns from your original dataset that you should transform). You can leave numerical features as-is, but you’ll need to take care of categorical columns using an appropriate encoding. Implement all steps (feature transforms and model training) in a single sklearn Pipeline.

Note: Both now and in Final Model Step, make sure to evaluate your model’s ability to generalize to unseen data!

There is no “required” performance metric that your baseline model needs to achieve.

In [253]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier

In [225]:
train_X, test_X, train_y, test_y = train_test_split(cleaned_data, cleaned_data['result'])

In [271]:
column_transformer = ColumnTransformer([
    ('kills', FunctionTransformer(), ['kills']),
    ('deaths', FunctionTransformer(), ['deaths']),
    ('assists', FunctionTransformer(), ['assists']),
])

pipeline = Pipeline([
    ('columns', column_transformer),
    ('log-reg', DecisionTreeClassifier(max_depth = 5)),
])

In [272]:
pipeline.fit(train_X, train_y)
predictions = pipeline.predict(test_X)

In [273]:
baseline_performance = np.mean(predictions == test_y)
baseline_performance

0.8679512609804477

### Final Model

Create a “final” model that improves upon the “baseline” model you created in Step 2. Do so by engineering at least two new features from the data, on top of any categorical encodings you performed in Baseline Model Step. (For instance, you may use a StandardScaler on a quantitative column and a QuantileTransformer transformer on a different column to get two new features.) Again, implement all steps in a single sklearn Pipeline. While deciding what features to use, you must perform a search for the best hyperparameters (e.g. tree depth) to use amongst a list(s) of options, either by using GridSearchCV or through some manual iterative method. In your notebook, state which hyperparameters you plan to tune and why before actually tuning them.

Optional: You are encouraged to try many different modeling algorithms for your final model (i.e. LinearRegression, RandomForestClassifier, Lasso, SVC, etc.) If you do this, make sure to clearly indicate in your notebook which model is your actual final model as that will be used to grade the above requirements.

Note 1: When training your model, make sure you use the same unseen and seen datasets from your baseline model. This way, the evaluation metric you get on your final model can be compared to your baseline’s on the basis of the model itself and not the dataset it was trained on. Based on which method you use for hyperparameter tuning, this may mean that you will need to use some of your training data as your validation data. If this is the case, make sure to train your final model on the whole dataset prior to evaluation.

Note 2: You will not be graded on “how much” your model improved from Baseline Model Step to Final Model Step. What you will be graded on is on whether or not your model improved, as well as your thoughtfulness and effort in creating features, along with the other points above.

In [216]:
cleaned_data.head(12)

,result,gameid,datacompleteness,playoffs,side,position,kills,deaths,assists,firstblood,gamelength,dragons,barons,elders,heralds,cspm,wpm,vspm,golddiffat15
0,False,ESPORTSTMNT01_2690210,True,False,Blue,top,2,3,2,0.0,1713,0.0,0.0,0.0,0.0,8.0911,0.2802,0.9107,391.0
1,False,ESPORTSTMNT01_2690210,True,False,Blue,jng,2,5,6,1.0,1713,0.0,0.0,0.0,0.0,5.1839,0.2102,1.6813,541.0
2,False,ESPORTSTMNT01_2690210,True,False,Blue,mid,2,2,3,0.0,1713,0.0,0.0,0.0,0.0,6.7601,0.6655,1.0158,-475.0
3,False,ESPORTSTMNT01_2690210,True,False,Blue,bot,2,4,2,1.0,1713,0.0,0.0,0.0,0.0,7.9159,0.4203,0.8757,-793.0
4,False,ESPORTSTMNT01_2690210,True,False,Blue,sup,1,5,6,1.0,1713,0.0,0.0,0.0,0.0,1.4711,1.0158,2.4168,443.0
5,True,ESPORTSTMNT01_2690210,True,False,Red,top,1,1,12,0.0,1713,0.0,0.0,0.0,0.0,8.0210,0.5254,1.0508,-391.0
6,True,ESPORTSTMNT01_2690210,True,False,Red,jng,4,1,10,0.0,1713,0.0,0.0,0.0,0.0,6.4098,0.2802,1.3660,-541.0
7,True,ESPORTSTMNT01_2690210,True,False,Red,mid,6,3,12,0.0,1713,0.0,0.0,0.0,0.0,7.5657,0.5254,1.0158,475.0
8,True,ESPORTSTMNT01_2690210,True,False,Red,bot,8,2,10,0.0,1713,0.0,0.0,0.0,0.0,11.1734,0.3152,1.4011,793.0
9,True,ESPORTSTMNT01_2690210,True,False,Red,sup,0,2,18,0.0,1713,0.0,0.0,0.0,0.0,1.0158,1.6112,2.3468,-443.0


In [267]:
column_transformer = ColumnTransformer([
    ('kills', StandardScaler(), ['kills']),
    ('deaths', StandardScaler(), ['deaths']),
    ('assists', StandardScaler(), ['assists']),
    ('position', OneHotEncoder(), ['position']),
    ('playoffs', OneHotEncoder(), ['playoffs']),
    ('gamelength', StandardScaler(), ['gamelength']),
    ('dragons', StandardScaler(), ['dragons']),
    ('barons', StandardScaler(), ['barons']),
    ('elders', FunctionTransformer(), ['elders']),
    ('heralds', StandardScaler(), ['heralds']),
    ('cspm', StandardScaler(), ['cspm']),
    ('vspm', StandardScaler(), ['vspm']),
    ('golddiff', StandardScaler(), ['golddiffat15']),
    ('xpdiff', StandardScaler(), ['xpdiffat15'])
])

pipeline = Pipeline([
    ('columns', column_transformer),
    ('log-reg', (LogisticRegression())),
])    

In [269]:
pipeline.fit(train_X, train_y)
predictions = pipeline.predict(test_X)

In [270]:
baseline_performance = np.mean(predictions == test_y)
baseline_performance

0.896130474481282